# 1.1 Validação de Entrada e Saída

Este notebook compara os dados do mês atual com o mês anterior para identificar:
- Novos beneficiários (Entraram)
- Beneficiários cancelados (Saíram)

**Entradas:**
- Planilha do Mês Atual (`dados_atualizados.xlsx` com novos beneficiários já cadastrados OU `dados.xlsx` com novos beneficiários já cadastrados)
- Planilha do Mês Anterior

**Saídas (Opcional):**
- `relatorio_entradas.xlsx`
- `relatorio_saidas.xlsx`

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### Carregamento dos Dados
Carregue a planilha atual e a planilha anterior para comparação.

In [ ]:
caminho_dados = '../Data/' # Caminho para a planilha de dados
mes_atual = '' # Mês Atual (Ex: 'JAN 2025')
mes_anterior = '' # Mês Anterior (Ex: 'FEV 2025')

dados_atual = pd.read_excel(caminho_dados, skiprows=1, index_col=0, sheet_name=mes_atual)
dados_anterior = pd.read_excel(caminho_dados, skiprows=1, index_col=0, sheet_name=mes_anterior) 

In [ ]:
print("Visualizando Mês Atual:")
display(dados_atual.head())
print("\nVisualizando Mês Anterior:")
display(dados_anterior.head())

### Função de Comparação
Lógica principal para cruzar os dados.

In [ ]:
def normaliza_nome(valor):
    if pd.isna(valor):
        return pd.NA
    texto = str(valor).strip()
    if texto == '' or texto.lower() == 'nan':
        return pd.NA
    return " ".join(texto.split())


def preparar_por_nome(df):
    df_limpo = df.copy()

    if 'NOME' in df_limpo.columns:
        df_limpo['NOME'] = df_limpo['NOME'].apply(normaliza_nome)
    if 'CPF' in df_limpo.columns:
        df_limpo['CPF'] = df_limpo['CPF'].astype('string').str.strip()
    if 'NASCIMENTO' in df_limpo.columns:
        df_limpo['NASCIMENTO'] = df_limpo['NASCIMENTO'].astype('string').str.strip()

    df_limpo = df_limpo[df_limpo['NOME'].notna() & (df_limpo['NOME'] != '')].copy()

    resumo = (
        df_limpo
        .groupby('NOME', as_index=False)
        .agg(
            CPF=('CPF', lambda s: s.dropna().iloc[0] if len(s.dropna()) else 'N/A'),
            NASCIMENTO=('NASCIMENTO', lambda s: s.dropna().iloc[0] if len(s.dropna()) else 'N/A'),
            HAS_MENSALIDADE=('MENSALIDADE', lambda s: s.notna().any())
        )
    )

    return resumo


def compare_months(df1, df2, month1_name, month2_name, gerar_df_entrada=False, gerar_df_saida=False):
    """
    Compara dois dataframes (meses) e identifica pessoas presentes em um mas não no outro,
    ou presentes em ambos com diferença na coluna 'MENSALIDADE' (NaN vs não NaN).
    """

    if df1 is None or df2 is None:
        print("Não é possível comparar: um dos dataframes está vazio.")
        return None, None

    df1_nomes = preparar_por_nome(df1)
    df2_nomes = preparar_por_nome(df2)

    df1_index = df1_nomes.set_index('NOME', drop=False)
    df2_index = df2_nomes.set_index('NOME', drop=False)

    nomes_df1 = df1_nomes['NOME'].dropna().unique()
    nomes_df2 = df2_nomes['NOME'].dropna().unique()

    missing_in_df2 = []
    for nome in nomes_df1:
        if nome not in nomes_df2:
            person_data = df1_index.loc[nome]
            missing_in_df2.append({
                'NOME': person_data.get('NOME', 'N/A'),
                'CPF': person_data.get('CPF', 'N/A'),
                'NASCIMENTO': person_data.get('NASCIMENTO', 'N/A')
            })

    missing_in_df1 = []
    for nome in nomes_df2:
        if nome not in nomes_df1:
            person_data = df2_index.loc[nome]
            missing_in_df1.append({
                'NOME': person_data.get('NOME', 'N/A'),
                'CPF': person_data.get('CPF', 'N/A'),
                'NASCIMENTO': person_data.get('NASCIMENTO', 'N/A')
            })

    entraram_status = []
    sairam_status = []

    common_nomes = list(set(nomes_df1) & set(nomes_df2))

    for nome in common_nomes:
        mensalidade_df1 = df1_index.loc[nome]['HAS_MENSALIDADE']
        mensalidade_df2 = df2_index.loc[nome]['HAS_MENSALIDADE']

        if not mensalidade_df1 and mensalidade_df2:
            person_data = df2_index.loc[nome]
            entraram_status.append({
                'NOME': person_data.get('NOME', 'N/A'),
                'CPF': person_data.get('CPF', 'N/A'),
                'NASCIMENTO': person_data.get('NASCIMENTO', 'N/A')
            })

        elif mensalidade_df1 and not mensalidade_df2:
            person_data = df1_index.loc[nome]
            sairam_status.append({
                'NOME': person_data.get('NOME', 'N/A'),
                'CPF': person_data.get('CPF', 'N/A'),
                'NASCIMENTO': person_data.get('NASCIMENTO', 'N/A')
            })

    lista_entrada = missing_in_df1 + entraram_status
    lista_saida = missing_in_df2 + sairam_status

    print("-" * 60)
    print(f"RELATÓRIO COMPARATIVO: {month1_name} vs {month2_name}")
    print("-" * 60)

    print(f"🟢 PESSOAS QUE ENTRARAM (Novos ou Reativados): {len(lista_entrada)}")
    if lista_entrada:
        for item in lista_entrada:
            print(f"   + {item['NOME']} | CPF: {item['CPF']}")
    else:
        print("   (Ninguém)")
    print("\n")

    print(f"🔴 PESSOAS QUE SAÍRAM (Cancelados ou Removidos): {len(lista_saida)}")
    if lista_saida:
        for item in lista_saida:
            print(f"   - {item['NOME']} | CPF: {item['CPF']}")
    else:
        print("   (Ninguém)")
    print("-" * 60)

    df_entrada = pd.DataFrame(lista_entrada) if gerar_df_entrada else None
    df_saida = pd.DataFrame(lista_saida) if gerar_df_saida else None

    if gerar_df_entrada and df_entrada is not None and not df_entrada.empty:
        df_entrada.to_excel('../Data/relatorio_entradas.xlsx', index=False)
        print("📁 Arquivo 'relatorio_entradas.xlsx' salvo na raiz.")

    if gerar_df_saida and df_saida is not None and not df_saida.empty:
        df_saida.to_excel('../Data/relatorio_saidas.xlsx', index=False)
        print("📁 Arquivo 'relatorio_saidas.xlsx' salvo na raiz.")

    return df_entrada, df_saida

### Executar Comparação

In [ ]:
if 'dados_anterior' in locals() and dados_anterior is not None:
    compare_months(
        df1=dados_anterior, 
        df2=dados_atual, 
        month1_name=mes_anterior, 
        month2_name=mes_atual,
        gerar_df_entrada=True,
        gerar_df_saida=True
    )
else:
    print("Não foi possível executar a comparação. Verifique o carregamento dos dados.")